In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install librosa

In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import regularizers
import os
from os import path

from tensorflow import keras
from keras.layers import Dense , Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences

from scipy.stats import pearsonr
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import pandas as pd

In [ ]:
cd '/content/drive/MyDrive/College/Grad Project/MIT/Audios_Answers'

/content/drive/.shortcut-targets-by-id/1vMfKGnS1lhSz-f1ELRTPdbnHyeg69WAI/Grad Project/MIT/Audios_Answers


In [ ]:
pretrained_model_path = '/content/drive/MyDrive/College/Grad Project/MIT/Pretrained Models/Emotion_Voice_Detection_Model.h5'

In [ ]:
pretrained_model = keras.models.load_model(pretrained_model_path)
print(len(pretrained_model.layers))
pretrained_model.summary()

18
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 216, 128)          768       
_________________________________________________________________
activation_8 (Activation)    (None, 216, 128)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 216, 128)          82048     
_________________________________________________________________
activation_9 (Activation)    (None, 216, 128)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 216, 128)          0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 27, 128)           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 27, 128)       

In [ ]:
def my_model():
  # new_pretrained_model = pretrained_model #keras.Sequential(pretrained_model.layers[:-1])   #pretrained_model
  new_pretrained_model = keras.Sequential(pretrained_model.layers[:-2])   #pretrained_model
  for layer in new_pretrained_model:
    layer.trainable = False
  
  new_pretrained_model.add(Dense(512 , activation='relu' , name='added_dense_1'))
  new_pretrained_model.add(Dropout(0.25 , name = 'added_dropout'))
  new_pretrained_model.add(Dense(1 , kernel_regularizer=keras.regularizers.l2(l2=0.01) , name='added_dense_2'))
  new_pretrained_model.compile(loss='mean_squared_error' , optimizer = 'adam')#optimizer=keras.optimizers.Adam(learning_rate=0.001))
  return new_pretrained_model

In [ ]:
estimator = KerasRegressor(build_fn=my_model , epochs=300 , batch_size = 32 , verbose=1 )

# PreProcessing Features

In [ ]:
def get_paths_list(identifier = 'P'):

  interview_paths = []
  for interview in range(90):
    for question in range(5):
      path_str = identifier + str(interview+1) + '/' + identifier + str(interview+1) + 'Q' + str(question+1) + '.wav'
      
      if path.exists(path_str):
        interview_paths.append(path_str)

  return interview_paths

In [ ]:
def get_all_paths():

  return get_paths_list('P') + get_paths_list('PP')

# getting features with librosa

In [ ]:
# df = pd.DataFrame(columns=['feature'])
# bookmark=0
# for index,y in enumerate(mylist):
#     if mylist[index][6:-16]!='01' and mylist[index][6:-16]!='07' and mylist[index][6:-16]!='08' and mylist[index][:2]!='su' and mylist[index][:1]!='n' and mylist[index][:1]!='d':
#         X, sample_rate = librosa.load('RawData/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
#         sample_rate = np.array(sample_rate)
#         mfccs = np.mean(librosa.feature.mfcc(y=X, 
#                                             sr=sample_rate, 
#                                             n_mfcc=13),
#                         axis=0)
#         feature = mfccs
#         #[float(i) for i in feature]
#         #feature1=feature[:135]
#         df.loc[bookmark] = [feature]
#         bookmark=bookmark+1

In [ ]:
df = pd.DataFrame(columns=['feature'])

In [ ]:
def preprocess_data_P():
  all_data = []
  all_paths = get_all_paths()
  i = 0
  for one_path in all_paths[:345]: #345/// [345:]
    i = i + 1
    # read wav signal and get its sequence and sampling rate
    data , sampling_rate = librosa.load(one_path)#librosa.load(one_path) #wav.read(one_path)
    sample_rate = np.array(sampling_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=13), axis=0)
    feature = mfccs
    df.loc[i] = [feature]
    # get the average of 2 channels of the audio
    # print(one_path)
    # print(signal)
    # print(signal.shape)
    # print(sampling_rate)

    # signal = np.mean(signal , axis=1)
    
    # pad sequences to be all in range of 55 seconds

    #final_signal = []
    #final_signal.append(signal)
    # pad function uses list of list not a single one
    #signal = pad_sequences(final_signal , maxlen=1350000)
    #signal = signal[0]

    # extract features
    #mfcc_features = mfcc(signal , sampling_rate , n_mfcc=27)
    #print(mfcc_features.shape)
    #chroma_features = chroma_stft(signal , sampling_rate)
    #print(chroma_features.shape)
    #mel_features = melspectrogram(signal , sampling_rate , n_mels = 128)
    #print(mel_features.shape)
    #contrast_features = spectral_contrast(signal , sampling_rate , n_bands =  6)
    #print(contrast_features.shape)
    #tonnetz_features = tonnetz(signal , sampling_rate)
    #print(tonnetz_features.shape)

    ##########################################################################################################################
    # log_mels = logfbank(signal , sampling_rate , winlen=0.025 , winstep=0.01 , nfilt=40 , nfft=1200)
    # deltas = delta(log_mels , 2)
    # deltas_deltas = delta(deltas , 2)
    ##########################################################################################################################

    #frame_features = np.concatenate((mfcc_features , chroma_features , mel_features , contrast_features , tonnetz_features) , axis = 0)
    #frame_features = np.transpose(frame_features)
    #frame_features = np.mean(frame_features , axis=0)
    #print(frame_features.shape)
    # append all features together
    ##########################################################################################################################
    # frame_features = np.stack((log_mels , deltas , deltas_deltas) , axis = 2)
    # frame_features = np.mean(frame_features , axis=0)
    ##########################################################################################################################
    # curr_noframes = frame_features.shape[0]
    # average_noframes = 2500
    # no_features = 180
    # padded_arr = np.zeros((average_noframes , no_features))

    # if average_noframes >= curr_noframes:
    #   start_idxpad = int((average_noframes - curr_noframes) / 2)
    #   padded_arr[ start_idxpad : start_idxpad + curr_noframes , :] = frame_features
    # else:
    #   random_idxs = np.random.randint(curr_noframes , size=average_noframes)
    #   padded_arr = frame_features[random_idxs , :]
    
    # frame_features = padded_arr
    # print(frame_features.shape)

    all_data.append(df)
    print('Done : ' + str(i))

  
  all_data = np.array(all_data)
  np.save('/content/drive/MyDrive/College/Grad Project/MIT/training of new pretrained models/input_features_librosa_P.npy' , all_data)
  print(all_data.shape)


In [ ]:
preprocess_data_P()

Done : 1
Done : 2
Done : 3
Done : 4
Done : 5
Done : 6
Done : 7
Done : 8
Done : 9
Done : 10
Done : 11
Done : 12
Done : 13
Done : 14
Done : 15
Done : 16
Done : 17
Done : 18
Done : 19
Done : 20
Done : 21
Done : 22
Done : 23
Done : 24
Done : 25
Done : 26
Done : 27
Done : 28
Done : 29
Done : 30
Done : 31
Done : 32
Done : 33
Done : 34
Done : 35
Done : 36
Done : 37
Done : 38
Done : 39
Done : 40
Done : 41
Done : 42
Done : 43
Done : 44
Done : 45
Done : 46
Done : 47
Done : 48
Done : 49
Done : 50
Done : 51
Done : 52
Done : 53
Done : 54
Done : 55
Done : 56
Done : 57
Done : 58
Done : 59
Done : 60
Done : 61
Done : 62
Done : 63
Done : 64
Done : 65
Done : 66
Done : 67
Done : 68
Done : 69
Done : 70
Done : 71
Done : 72
Done : 73
Done : 74
Done : 75
Done : 76
Done : 77
Done : 78
Done : 79
Done : 80
Done : 81
Done : 82
Done : 83
Done : 84
Done : 85
Done : 86
Done : 87
Done : 88
Done : 89
Done : 90
Done : 91
Done : 92
Done : 93
Done : 94
Done : 95
Done : 96
Done : 97
Done : 98
Done : 99
Done : 100
Done : 1

In [ ]:
def preprocess_data_PP():
  all_data = []
  all_paths = get_all_paths()
  i = 0
  for one_path in all_paths[345:]: #345/// [345:]
    i = i + 1
    # read wav signal and get its sequence and sampling rate
    data , sampling_rate = librosa.load(one_path)#librosa.load(one_path) #wav.read(one_path)
    sample_rate = np.array(sampling_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=13), axis=0)
    feature = mfccs
    df.loc[i] = [feature]
    # get the average of 2 channels of the audio
    # print(one_path)
    # print(signal)
    # print(signal.shape)
    # print(sampling_rate)

    # signal = np.mean(signal , axis=1)
    
    # pad sequences to be all in range of 55 seconds

    #final_signal = []
    #final_signal.append(signal)
    # pad function uses list of list not a single one
    #signal = pad_sequences(final_signal , maxlen=1350000)
    #signal = signal[0]

    # extract features
    #mfcc_features = mfcc(signal , sampling_rate , n_mfcc=27)
    #print(mfcc_features.shape)
    #chroma_features = chroma_stft(signal , sampling_rate)
    #print(chroma_features.shape)
    #mel_features = melspectrogram(signal , sampling_rate , n_mels = 128)
    #print(mel_features.shape)
    #contrast_features = spectral_contrast(signal , sampling_rate , n_bands =  6)
    #print(contrast_features.shape)
    #tonnetz_features = tonnetz(signal , sampling_rate)
    #print(tonnetz_features.shape)

    ##########################################################################################################################
    # log_mels = logfbank(signal , sampling_rate , winlen=0.025 , winstep=0.01 , nfilt=40 , nfft=1200)
    # deltas = delta(log_mels , 2)
    # deltas_deltas = delta(deltas , 2)
    ##########################################################################################################################

    #frame_features = np.concatenate((mfcc_features , chroma_features , mel_features , contrast_features , tonnetz_features) , axis = 0)
    #frame_features = np.transpose(frame_features)
    #frame_features = np.mean(frame_features , axis=0)
    #print(frame_features.shape)
    # append all features together
    ##########################################################################################################################
    # frame_features = np.stack((log_mels , deltas , deltas_deltas) , axis = 2)
    # frame_features = np.mean(frame_features , axis=0)
    ##########################################################################################################################
    # curr_noframes = frame_features.shape[0]
    # average_noframes = 2500
    # no_features = 180
    # padded_arr = np.zeros((average_noframes , no_features))

    # if average_noframes >= curr_noframes:
    #   start_idxpad = int((average_noframes - curr_noframes) / 2)
    #   padded_arr[ start_idxpad : start_idxpad + curr_noframes , :] = frame_features
    # else:
    #   random_idxs = np.random.randint(curr_noframes , size=average_noframes)
    #   padded_arr = frame_features[random_idxs , :]
    
    # frame_features = padded_arr
    # print(frame_features.shape)

    all_data.append(df)
    print('Done : ' + str(i))

  
  all_data = np.array(all_data)
  np.save('/content/drive/MyDrive/College/Grad Project/MIT/training of new pretrained models/input_features_librosa_PP.npy' , all_data)
  print(all_data.shape)


# DATA

In [ ]:
cd '/content/drive/MyDrive/College/Grad Project/MIT/Audios_Answers'

/content/drive/.shortcut-targets-by-id/1vMfKGnS1lhSz-f1ELRTPdbnHyeg69WAI/Grad Project/MIT/Audios_Answers


In [ ]:
def get_labels():
  labels_path = '/content/drive/MyDrive/College/Grad Project/MIT/turker_scores_full_interview.csv'
  data_frame = pd.read_csv(labels_path)
  data_frame = data_frame[data_frame['Worker'] == 'AGGR']
  return data_frame

In [ ]:
def set_data(trait = 'Overall'):
  
  all_interviews = np.load('/content/drive/MyDrive/College/Grad Project/MIT/interview_features.npy')
  print(all_interviews.shape)
  #all_interviews = all_interviews.reshape(all_interviews.shape[0] , all_interviews.shape[1] * all_interviews.shape[2])
  #padded_array = np.zeros((all_interviews.shape[0] , 180))
  #padded_array[: , :120] = all_interviews
  #all_interviews = padded_array
  #print(all_interviews.shape)
  labels = get_labels()[trait]
  

  P_data , PP_data , P_labels , PP_labels = all_interviews[0:69 , : ] , all_interviews[69: , : ] , labels[0:69] , labels[69:]
  P_data , PP_data , P_labels , PP_labels = shuffle(P_data , PP_data , P_labels , PP_labels , random_state = 2)

  del(all_interviews)
  del(labels)

  train_data , test_data = np.concatenate((P_data[0:55 , :] , PP_data[0:55 , :])) , np.concatenate((P_data[55: , :] , PP_data[55: , :]))
  train_labels, test_labels = np.concatenate((P_labels[0:55] , PP_labels[0:55])), np.concatenate((P_labels[55:] , PP_labels[55:]))

  return train_data , test_data , train_labels , test_labels

In [ ]:
def training_SVR(trait = 'Overall'):
  
  train_data , test_data , train_labels , test_labels = set_data(trait)
  print("train_data: ", train_data.shape)
  print("test_data: ", test_data.shape)

  train_data = train_data.reshape(train_data.shape[0] , train_data.shape[1] * train_data.shape[2])
  test_data = test_data.reshape(test_data.shape[0] , test_data.shape[1] * test_data.shape[2])

  scaler = StandardScaler()
  scaler.fit(train_data)
  train_data = scaler.transform(train_data)
  test_data = scaler.transform(test_data)

  train_data = train_data.reshape(train_data.shape[0] , 1 , 180)
  test_data = test_data.reshape(test_data.shape[0] , 1, 180)
  
  #val_data , val_labels = test_data[:14 ,:] , test_labels[:14]
  #test_data , test_labels = test_data[14: ,:] , test_labels[14:]

  #param_grid = [{'kernel': ['linear' , 'rbf']}]
  #params = {'batch_size': [16 , 32]} 
  #        'epochs': [50, 100]}

  grids = estimator
  #grids = GridSearchCV(estimator , params,cv=10)

  grids.fit(train_data , train_labels) #, validation_data = (val_data , val_labels))
  print(grids.score(test_data , test_labels))
  Y_predicted = grids.predict(test_data)
  # Y_predicted = np.mean(Y_predicted , axis=1)
  correlation_res , _ = pearsonr(Y_predicted , test_labels)
  print('--------------------------------' + trait + '--------------------------------')
  print(correlation_res)
  print(test_labels)
  print(Y_predicted)

  plt.scatter(Y_predicted , test_labels)

  return grids  

In [ ]:
pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp37-none-any.whl size=5887 sha256=d306b90307043e0ee428e504602ef22a0b48f5c63ba86f0a7cf5db79f0dd137e
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [ ]:
from tensorflow import keras
from keras.layers import Dense , Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from python_speech_features import logfbank , delta
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences

from scipy.stats import pearsonr
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle

In [ ]:
pretrained_model_path = '/content/drive/MyDrive/College/Grad Project/MIT/Pretrained Models/Emotion_Voice_Detection_Model.h5'

In [ ]:
pretrained_model = keras.models.load_model(pretrained_model_path)
print(len(pretrained_model.layers))
pretrained_model.summary()

18
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 216, 128)          768       
_________________________________________________________________
activation_8 (Activation)    (None, 216, 128)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 216, 128)          82048     
_________________________________________________________________
activation_9 (Activation)    (None, 216, 128)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 216, 128)          0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 27, 128)           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 27, 128)       

In [ ]:
#pretrained_model.layers.pop()
#print(len(pretrained_model.layers))
#pretrained_model.summary()

In [ ]:
def my_model():
  new_pretrained_model = pretrained_model #keras.Sequential(pretrained_model.layers[:-1])   #pretrained_model
  for layer in new_pretrained_model.layers[:-1]:
    layer.trainable = False
  
  # new_pretrained_model.add(Dense(16 , activation='relu' , name='aaaa'))
  # new_pretrained_model.add(Dropout(0.25 , name = 'dddd'))
  #new_pretrained_model.add(Dense(1 , kernel_regularizer=keras.regularizers.l2(l2=0.01) , name='bbb'))
  new_pretrained_model.compile(loss='mean_squared_error' , optimizer = 'adam')#optimizer=keras.optimizers.Adam(learning_rate=0.001))
  return new_pretrained_model

In [ ]:
estimator = KerasRegressor(build_fn=my_model , epochs=300 , batch_size = 32 , verbose=1 )

# Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
import os
from os import path

In [ ]:
cd '/content/drive/MyDrive/College/Grad Project/MIT/Audios_Answers'

/content/drive/.shortcut-targets-by-id/1vMfKGnS1lhSz-f1ELRTPdbnHyeg69WAI/Grad Project/MIT/Audios_Answers


In [ ]:
def get_labels():
  labels_path = '/content/drive/MyDrive/College/Grad Project/MIT/turker_scores_full_interview.csv'
  data_frame = pd.read_csv(labels_path)
  data_frame = data_frame[data_frame['Worker'] == 'AGGR']
  return data_frame

In [ ]:
def set_data(trait = 'Overall'):
  
  all_interviews = np.load('/content/drive/MyDrive/College/Grad Project/MIT/interview_features.npy')
  print(all_interviews.shape)
  #all_interviews = all_interviews.reshape(all_interviews.shape[0] , all_interviews.shape[1] * all_interviews.shape[2])
  #padded_array = np.zeros((all_interviews.shape[0] , 180))
  #padded_array[: , :120] = all_interviews
  #all_interviews = padded_array
  #print(all_interviews.shape)
  labels = get_labels()[trait]
  

  P_data , PP_data , P_labels , PP_labels = all_interviews[0:69 , : ] , all_interviews[69: , : ] , labels[0:69] , labels[69:]
  P_data , PP_data , P_labels , PP_labels = shuffle(P_data , PP_data , P_labels , PP_labels , random_state = 2)

  del(all_interviews)
  del(labels)

  train_data , test_data = np.concatenate((P_data[0:55 , :] , PP_data[0:55 , :])) , np.concatenate((P_data[55: , :] , PP_data[55: , :]))
  train_labels, test_labels = np.concatenate((P_labels[0:55] , PP_labels[0:55])), np.concatenate((P_labels[55:] , PP_labels[55:]))

  return train_data , test_data , train_labels , test_labels

In [ ]:
def training_SVR(trait = 'Overall'):
  
  train_data , test_data , train_labels , test_labels = set_data(trait)
  print("train_data: ", train_data.shape)
  print("test_data: ", test_data.shape)

  train_data = train_data.reshape(train_data.shape[0] , train_data.shape[1] * train_data.shape[2])
  test_data = test_data.reshape(test_data.shape[0] , test_data.shape[1] * test_data.shape[2])

  scaler = StandardScaler()
  scaler.fit(train_data)
  train_data = scaler.transform(train_data)
  test_data = scaler.transform(test_data)

  train_data = train_data.reshape(train_data.shape[0] , 1 , 180)
  test_data = test_data.reshape(test_data.shape[0] , 1, 180)
  
  #val_data , val_labels = test_data[:14 ,:] , test_labels[:14]
  #test_data , test_labels = test_data[14: ,:] , test_labels[14:]

  #param_grid = [{'kernel': ['linear' , 'rbf']}]
  #params = {'batch_size': [16 , 32]} 
  #        'epochs': [50, 100]}

  grids = estimator
  #grids = GridSearchCV(estimator , params,cv=10)

  grids.fit(train_data , train_labels) #, validation_data = (val_data , val_labels))
  print(grids.score(test_data , test_labels))
  Y_predicted = grids.predict(test_data)
  # Y_predicted = np.mean(Y_predicted , axis=1)
  correlation_res , _ = pearsonr(Y_predicted , test_labels)
  print('--------------------------------' + trait + '--------------------------------')
  print(correlation_res)
  print(test_labels)
  print(Y_predicted)

  plt.scatter(Y_predicted , test_labels)

  return grids  

In [ ]:
overall = training_SVR('Overall')
# overall.save("AHNPS-c-LSTM_overall.h5")

(138, 2500, 180)
train_data:  (110, 2500, 180)
test_data:  (28, 2500, 180)


ValueError: ignored

In [ ]:
overall.save("AHNPS-c-LSTM_overall.h5")

AttributeError: ignored